In [1]:
# 1.bibliotheck laden
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.keras import Model, layers
from tensorflow.keras.datasets import mnist


In [2]:
# 2.Input parameters or variables 

num_classes = 10  # 0-9 num classes
num_feature = 784 # data features  image shape 28*28

learning_rate = 0.01
training_steps =100
batch_size = 56
display_step=5

n_hidden_1=128
n_hidden_2=256

In [3]:
#3. Dataset laden vom mnist

(x_train,y_train),(x_test,y_test)=mnist.load_data()

# conver to float 32

x_train,x_test = np.array( x_train,np.float32),np.array( x_test,np.float32)

# flatten image

x_train,x_test = x_train.reshape([-1,num_feature]),x_test.reshape([-1,num_feature])

# Normalize

x_train,x_test = x_train/255.,x_test/255.

In [4]:
print(x_train[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [5]:
print(y_train[0])

5


In [6]:
# 4 Data shuffle

training_data = tf.data.Dataset.from_tensor_slices((x_train,y_train))

training_data = training_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
# training_data.dtype

In [7]:
# 5 Neural net work model with feed forward.
class FD(Model):
    # Set layers.
    def __init__(self):
        super(FD, self).__init__()
        # First fully-connected hidden layer.
        self.fc1 = tf.keras.layers.Dense(n_hidden_1, activation=tf.nn.relu)
        # Second fully-connected hidden layer.
        self.fc2 = tf.keras.layers.Dense(n_hidden_2, activation=tf.nn.relu)
        # final fully-connecter hidden layer.
        self.out = tf.keras.layers.Dense(num_classes)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build neural network model.
HW_NN = FD()
        

In [8]:
# 6 loss funtion

def loss_fn(x,y): 
     # converting labels into int64 for cross entropy funtion
    
    y = tf.cast(y,tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=x)
    loss = tf.reduce_mean(loss)
    return loss
   

In [9]:
# 7 accuracy funtion
def acc_fn(y_pred,y_act):
    correct_pred=tf.equal(tf.argmax(y_pred,1),tf.cast(y_act,tf.int64))
    accuracy= tf.reduce_mean(tf.cast(correct_pred,tf.float32),axis=-1 )
    return accuracy
    
    

In [10]:
# 8 optimization 
#def optim_fn( learning_rate):
optimizer = tf.optimizers.SGD(learning_rate)
    

In [11]:
# 9 run_optimization process

def run_optimization(x,y):
    with tf.GradientTape() as g:
        # Forward pass
        pred = HW_NN(x,is_training=True)
        loss =loss_fn(pred,y)
        trainable_variables = HW_NN.trainable_variables
        # Compute gradients
        gradients=g.gradient(pred, trainable_variables)
        
        optimizer.apply_gradients(zip(gradients,trainable_variables))
    


In [12]:
# 10 no of trianing steps and display control

for step, (batch_x,batch_y) in enumerate( training_data.take(training_steps), 1):
    run_optimization(batch_x,batch_y)
    
    
    if step % display_step == 0:
        
        pred = HW_NN(batch_x,is_training=True)
        loss =loss_fn(pred,batch_y)
        accuracy = acc_fn( pred,batch_y)
        print( "step : %i, loss: %f,accuracy: %f " %(step,loss,accuracy))
        
        
        

step : 5, loss: nan,accuracy: 0.142857 
step : 10, loss: nan,accuracy: 0.107143 
step : 15, loss: nan,accuracy: 0.107143 
step : 20, loss: nan,accuracy: 0.071429 
step : 25, loss: nan,accuracy: 0.089286 
step : 30, loss: nan,accuracy: 0.053571 
step : 35, loss: nan,accuracy: 0.142857 
step : 40, loss: nan,accuracy: 0.107143 
step : 45, loss: nan,accuracy: 0.107143 
step : 50, loss: nan,accuracy: 0.107143 
step : 55, loss: nan,accuracy: 0.071429 
step : 60, loss: nan,accuracy: 0.089286 
step : 65, loss: nan,accuracy: 0.142857 
step : 70, loss: nan,accuracy: 0.089286 
step : 75, loss: nan,accuracy: 0.107143 
step : 80, loss: nan,accuracy: 0.089286 
step : 85, loss: nan,accuracy: 0.089286 
step : 90, loss: nan,accuracy: 0.125000 
step : 95, loss: nan,accuracy: 0.178571 
step : 100, loss: nan,accuracy: 0.071429 
